In [2]:
import requests
from xml.etree import ElementTree
from datetime import datetime
from pathlib import Path

In [22]:
BASE_CATALOG_URL = "https://esgf-data.dwd.de/thredds/catalog/esgf3/data/climatepredictionsde/seasonal/output/public/DE-0075x005/DWD/GCFS21/svhYYYY{month:02}01/sfc{year}{month:02}01/{scenario}/DWD-EPISODES2022/v1-r1/day/{variable}/"
SCENARIOS = ['r1i1p1', 'r2i1p1', 'r3i1p1']
VARIABLES = ['hurs', 'pr', 'psl', 'rsds', 'sfcWind', 'tas', 'tasmax', 'tasmin']

In [60]:
def get_download_url(year, month, scenario, variable):
    """Get the latest catalog URL for the specified year, month, and scenario."""

    # get the version folder's name
    catalog_url = f"{BASE_CATALOG_URL.format(year=year, month=month, scenario=scenario,variable=variable)}catalog.xml"
    catalog = requests.get(catalog_url)
    catalog_tree = ElementTree.fromstring(catalog.content)
    namespace = {"thredds": "http://www.unidata.ucar.edu/namespaces/thredds/InvCatalog/v1.0"}
    catalog = catalog_tree.findall(".//thredds:catalogRef", namespace)
    latest_versions = []
    for catalog_ref in catalog:
        latest_versions.append(catalog_ref.attrib['name'])
    latest_version = max(latest_versions)

    # compose catalog url for the latest version
    latest_version_url = f"{BASE_CATALOG_URL.format(year=year, month=month, scenario=scenario,variable=variable)}{latest_version}/catalog.xml"
    
    # Get the dataset name/ urlPath
    dataset_name_reponse = requests.get(latest_version_url)
    dataset_name_catalog_tree = ElementTree.fromstring(dataset_name_reponse.content)
    dataset_name_catalog = dataset_name_catalog_tree.findall(".//thredds:dataset", namespace)
    dataset_path = ''
    for dataset in dataset_name_catalog:
        if dataset.attrib.get('urlPath'):
            dataset_path = dataset.attrib['urlPath']

    https_download_url = f"https://esgf-data.dwd.de/thredds/fileServer/{dataset_path}"
    return https_download_url

In [61]:
p = get_download_url(2025, 1, 'r1i1p1', 'hurs')

In [4]:
def get_local_path():
    """Get the base local path for storing NetCDF forecast files."""
    local = Path(__file__).resolve().parent.parent
    return local / 'climate_netcdf_forecast'

In [8]:
def download_and_save_nc_file(nc_url, save_path):
    """Download and save the NetCDF file to the specified local path."""
    response = requests.get(nc_url)
    response.raise_for_status()

    filename = nc_url.split("/")[-1]
    save_path = save_path / filename
    save_path.parent.mkdir(parents=True, exist_ok=True)

    with open(save_path, "wb") as file:
        file.write(response.content)

    print(f"Downloaded: {filename} to {save_path}")

In [10]:
now = datetime.now()
year = now.year
month = now.month

